In [3]:
# !pip install langchain==0.1.13 langchainhub==0.1.15 python-dotenv==1.0.1 huggingface_hub==0.22.2
# !pip install sentence-transformers
# !pip install groq
# !pip install langchain_groq
# !pip install google-search-results
# !pip install faiss-cpu

In [22]:
from langchain.agents import load_tools
from groq import Groq
import string
import os
import random
api_keys = ['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','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',
            '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','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']

client = Groq(
    
    api_key = random.choice(api_keys)
)

from langchain_groq import ChatGroq
from langchain.agents import initialize_agent
llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192")



In [5]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system","You are a world class technical documentation writer"
        ),
        (
            "user","{input}"
        )
    ]
)


chain = prompt | llm


chain.invoke("how cna langsmith help with tetsing")

AIMessage(content="As a technical documentation writer, I'd be happy to help explain how Langsmith can assist with testing.\n\n**What is Langsmith?**\nLangsmith is an innovative AI-powered tool that enables natural language processing (NLP) and machine learning (ML) capabilities to analyze and understand human language. In the context of testing, Langsmith can be a game-changer.\n\n**How can Langsmith help with testing?**\n\n1. **Automated Test Case Generation**: Langsmith can analyze requirements documents, user stories, or technical specifications to automatically generate test cases, reducing the time and effort required for test case creation.\n2. **Test Data Generation**: Langsmith can generate test data, such as input values, expected results, and edge cases, to support testing efforts.\n3. **Test Script Generation**: Langsmith can create test scripts in various programming languages, such as Python, Java, or C++, to streamline testing processes.\n4. **Test Case Prioritization**:

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser
chain.invoke({"input":"how can langsmith help with testing"})

"Langsmith, a remarkable tool for natural language processing (NLP) and machine learning (ML) model development! As a technical documentation writer, I'd be delighted to explain how Langsmith can assist with testing.\n\n**Automated Testing for NLP Models**\n\nLangsmith can help with testing in several ways:\n\n1. **Test Data Generation**: Langsmith can generate high-quality, diverse, and relevant test data for NLP models. This data can be used to test models for accuracy, robustness, and edge cases, ensuring they perform well in real-world scenarios.\n2. **Model Evaluation**: Langsmith provides a platform to evaluate NLP models on various metrics, such as accuracy, F1-score, and ROUGE score. This enables developers to assess their models' performance and identify areas for improvement.\n3. **Edge Case Detection**: Langsmith's advanced NLP capabilities can help identify edge cases that might not be caught through traditional testing methods. This ensures that NLP models are robust and c

In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

d:\DEV\WebdevFolder\RealEstateAI\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
vector = FAISS.from_documents(documents,embeddings)


from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

{context}


Question: {input}""")

document_chain = create_stuff_documents_chain(llm,prompt)


from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()

retrieval_chain = create_retrieval_chain(retriever, document_chain)


response = retrieval_chain.invoke({"input":"how can langsmith help with testing"})
print(response["answer"])

Based on the provided context, LangSmith can help with testing in the following ways:

1. **Evaluation**: LangSmith allows you to evaluate your LLM application by creating a dataset, defining an evaluator, and running an experiment to test the performance of your AI system.
2. **Tracing**: LangSmith provides a way to log traces of your LLM application, which can be used to monitor and debug your application. This can help identify issues and improve the testing process.
3. **Automated testing**: LangSmith provides a platform for building production-grade LLM applications, which implies that it can be used to automate testing of these applications.

Overall, LangSmith seems to provide a comprehensive platform for building, testing, and evaluating LLM applications.


In [10]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system","You are a world class technical documentation writer"
        ),
        (
            "user","{input}"
        )
    ]
)


from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),
    ("user","Given the above conversation, generate a search query to look up in order to get information")
])

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)


from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can Langsmith help test my LLm applications?"),AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history":chat_history,
    "input":"Tell me how"
})

[Document(page_content="Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith\u200bPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key\u200bTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment\u200bShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace\u200bWe provide multiple ways to log traces to LangSmith. Below, we'll highlight", metadata={'source': 'https://doc

In [11]:
prompt = ChatPromptTemplate.from_messages([
    ("system","Answer the user's questions based on the below context\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user","{input}"),

])
document_chain = create_stuff_documents_chain(llm,prompt)
retrieval_chain = create_retrieval_chain(retriever_chain,document_chain)


from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can Langsmith help test my LLm applications?"),AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history":chat_history,
    "input":"Tell me how"
})

{'chat_history': [HumanMessage(content='Can Langsmith help test my LLm applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content="Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith\u200bPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key\u200bTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment\u200bShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your

In [12]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for info about LangSmith. For any questions about LangSmith, use this tool!"
)


from langchain_community.tools.tavily_search import TavilySearchResults


os.environ['TAVILY_API_KEY'] = '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'


search = TavilySearchResults(tavily_api_key = os.getenv('TAVILY_API_KEY'))

# from langchain.agents import Tool, initialize_agent
# from langchain import LLMMathChain
# llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
# math_tool = Tool.from_function(
#         func=llm_math_chain.run,
#         name="Calculator",
#         description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions.",
#     )

tools = [search]

In [13]:
from langchain_groq import ChatGroq
from langchain import hub
from langchain.agents import create_structured_chat_agent
from langchain.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, ChatMessagePromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, PromptTemplate
import typing
import langchain_core
#prompt = hub.pull("hwchase17/structured-chat-agent")
prompt = ChatPromptTemplate(
    input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'],
    input_types={
        'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]
        },
    metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'structured-chat-agent', 'lc_hub_commit_hash': 'ea510f70a5872eb0f41a4e3b7bb004d5711dc127adee08329c664c6c8be5f13c'},
    messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['tool_names', 'tools'],
    template='Respond to the human as helpfully and accurately as possible. BE AS SPECIFIC AND DETAILED AS POSSIBLE. Your answer must be long and you must perform a giant web search, that is, not superficial. You have access to the following tools:\n\n{tools}\n\nUse a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).\n\nValid "action" values: "Final Answer" or {tool_names}\n\nProvide only ONE action per \
JSON_BLOB\n```\nObservation: action result\n... (repeat Thought/Action/Observation N times)\nThought: I know what to respond\nAction:\n```\n{{\n  "action": "Final Answer",\n  "action_input": "Final response to human"\n}}\n\nBegin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Your answer must be long and you need to perform enough searches with the searching tool to get enough information. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation. Please put all your observations into the final answer IT MUST BE DETAILED FINAL ANSWER')),
    MessagesPlaceholder(variable_name='chat_history', optional=True),
    HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'],
    template='{input}\n\n{agent_scratchpad}\n (reminder to respond in a JSON blob no matter what)'))]
)

agent = create_structured_chat_agent(llm,tools,prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:
system_msg = """
                You are an expert at questioning the client about their renovation or contstruction project quote.
                Please keep in mind all of these things when asking questions: ask the user specific info needed for the quote, such as:

                    1.Property Details:
                    Address and location of the property.
                    Type of property (e.g., residential, commercial).
                    Age and current condition of the property.
                    Square footage of the area to be renovated.
                    2.Scope of the Project:
                    Detailed description of the renovation work required.
                    Specific areas or rooms that need renovation.
                    Desired changes and additions (e.g., new rooms, extensions, wall removal).
                    Level of finishes desired (standard, premium, luxury).
                    3.Design Preferences:
                    Style or theme of the renovation (modern, traditional, industrial, etc.).
                    Color schemes and material preferences.
                    Any specific fixtures or features desired (e.g., type of flooring, lighting fixtures).
                    4.Budget:
                    Your budget range for the project.
                    Any flexibility within the budget.
                    5.Timeline:
                    Desired start date and completion date.
                    Any constraints or deadlines (e.g., events planned at the property).
                    Legal and Compliance Information:
                    Information on any permits or approvals already obtained.
                    Any restrictions or caveats affecting the property.
                    6.Utility Modifications:
                    Changes or improvements to electrical, plumbing, or HVAC systems.
                    Details of any provisions for appliances or systems (e.g., energy-efficient or smart home features).
                    7.Access & Logistics:
                    Access requirements for renovation or construction work (e.g., elevator access, restricted hours).
                    Logistic considerations necessary for the project (e.g., space for storing materials, parking for construction vehicles).
                    8.Special Requirements:
                    Access provisions for renovations or construction work done on the property.
                    Special considerations or instructions (e.g., conserving existing architectural features).
                    What specific materials they are looking at.
                    Information about other construction or renovation projects completed on the property.
                    IMPORTANT!: Make sure you get clear answers from the user that can actually be used for making the quote, and if you are not provided a clear answer ask to clarify, and restate the question and explain what it means.
                    IMPORTANT!: Ask each question that you have ONE BY ONE

                    IMPORTANT!: WHEN YOU HAVE ALL THE INFORMATION, JUST SAY 'hehehe' at the end.

                """

client.api_key = random.choice(api_keys)
chat_history = []
while False:


    client_req = str(input())
    quotebot = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": system_msg

            },

            {
                "role": "user",
                "content": client_req + "Chat History for your own context and info - DONT ASK A QUESTION IF THE ANSWER IS ALREADY IN CHAT HISTORY: " + str(chat_history)
            }
        ],

        model="llama3-70b-8192",
    )
    output = (quotebot.choices[0].message.content)
    print(output)
    chat_history.append(output)
    if('hehehe' in output.lower()): break

In [19]:
chat_history.append("""Let's get started. I'll ask the questions one by one to ensure I get all the necessary information for the renovation or construction project quote.

Here's my first question:

What is the address and location of the property you'd like to renovate or construct?
13582 cobra drive Herndon VA
It seems like we've already discussed the address and location of the property, which is 13582 Cobra Drive, Herndon, VA. Let's move on to the next question!

What type of property is this? Is it a residential or commercial property?
residential
Since we already have the address and location, let's move on to the next question.

You mentioned it's located at 13582 Cobra Drive, Herndon, VA. Now, you've already told me the property type, which is residential.

Let's proceed to the next question.

Can you please tell me the age and current condition of the property? For example, is it an old house that needs a lot of work, or is it relatively new and well-maintained?
It was built in 1990 and is in fairly good condition. I just want to renovate 25% of my basement.
Since you've already mentioned the age and condition of the property, I'll move on to the next question.

What is the square footage of the area to be renovated, which is 25% of your basement?
around 150 square feet
It seems like we've already covered some essential details about the property. Let's move on to the scope of the project.

Can you please provide a detailed description of the renovation work required for the 150 square feet area in the basement? For example, do you need to redo the flooring, install new walls, add lighting, or make any other specific changes?
yes, I want to add nice wooden flooring, bright overhead lighting and have shelves lining one side of the wall.
I see you've already provided some valuable information about your project. Based on your previous responses, I understand that you want to renovate a 150 sq ft area in your basement, including adding nice wooden flooring, bright overhead lighting, and shelves lining one side of the wall.

Let's dive deeper into the scope of the project. Can you please specify which areas or rooms within the 150 sq ft space require renovation? For instance, do you need to renovate the entire space, or are there specific areas, like a home office or a entertainment corner, that need more attention?
the entire space is the entertainment space
I see that you've already provided some excellent details about your project. Next, I'd like to know more about your desired level of finishes for this renovation. Are you looking for standard, premium, or luxury finishes for the 150 sq ft area in your basement?
premium features, like a 100 inch TV, a beautiful mirror on one side of the wall, nice large couchm, a games area for a pool table and a fooseball table and on the far end it will go down and there will be a tinted glass wall and behind it there will be a hot tub.
Based on our conversation, I understand that you want to renovate a 150 sq ft area in your basement with premium features like a 100-inch TV, a beautiful mirror, a large couch, a game area with a pool table and foosball table, and a hot tub behind a tinted glass wall.

Let's move on to the next question. Can you please tell me more about your design preferences for this renovation? For example, are you leaning towards a modern, traditional, industrial, or any other style for the renovation?
modern
It seems like we've already discussed the design preferences, and you're leaning towards a modern style for the renovation. Let's proceed to the next question.

What is your budget range for this project? Are you looking to spend a specific amount, and is there any flexibility within that budget?
500,000 dollars
It seems like we've already discussed the budget, and you mentioned $500,000. Let's move on to the next question!

Can you please tell me about your desired timeline for this project? Do you have a specific start date and completion date in mind, or are there any constraints or deadlines you're working with?
yes I need it done within 2 months at most
I see you mentioned earlier that you need the project completed within 2 months at most. Can you please confirm if there are any specific events or deadlines that might be driving this timeline, and whether there's any flexibility in case of unexpected delays?
yeah there is no specific even I just want it really badly
It seems like we've already discussed quite a bit about your project. Let's move on to the next question.

Can you please provide information on any permits or approvals you've already obtained for this renovation project, and are there any restrictions or caveats that might affect the property?
its my property so I can do what I want
Since we've already covered the necessary information about the property and scope of the project, let's proceed to the utility modifications.

Can you please tell me about any changes or improvements you'd like to make to the electrical, plumbing, or HVAC systems in the 150 sq ft area of your basement during this renovation?
yeah just increase the amount of vents in the room and run more electricity in there as well I guess for all the appliances
Based on our conversation, I understand that you want to increase the number of vents in the room and run more electricity for the appliances.

Let's get back to the questions. Since we've already discussed utility modifications, let's move on to the next question.

Can you please provide details on any access requirements or logistic considerations necessary for this renovation project? For example, are there any restrictions on working hours, elevator access, or parking for construction vehicles?
yes, its a neighborhood so you cant bring an entire crane and a bunch of trucks around, and you cannot work during the night
Based on our conversation, I understand that you've already provided some important details about the property and scope of the project. Let's dive deeper into the access and logistics considerations.

Can you please tell me more about the access requirements for the renovation work, specifically regarding the restricted hours and neighborhood constraints? For instance, are there any specific times when workers can access the property, or are there any limitations on the number of vehicles that can be parked nearby?
yes from 6pm onwards they cannot work
Let's continue from where we left off.

Since we've already discussed the access requirements and logistic considerations, let's move on to the special requirements.

Can you please tell me about any special provisions or instructions for this renovation project, such as conserving existing architectural features or specific materials you're interested in using?
worker can come in from 9am to 6pm
Let's proceed to the next question.
ok
Based on our conversation, I understand that you've already provided some valuable information about your project. Since we've already covered the necessary information about the property, scope of the project, design preferences, budget, timeline, permits, utility modifications, access, and logistics, let's move on to the next question.

Can you please tell me more about any other special considerations or instructions for this renovation project, such as specific materials you'd like to use, or any other unique requirements that might impact the project?

(Please answer this question, and I'll proceed to the next one. If you've already mentioned something related to this question, please clarify or provide more details.)
not really I just have an idea of kind of a modern theme
Let's continue from where we left off. Since we've already covered the necessary information about the property, scope of the project, design preferences, budget, timeline, permits, utility modifications, access, and logistics, let's move on to the next question.

Can you please tell me more about any previous construction or renovation projects completed on the property, and how those might impact this current project?
no nothing
Let's continue from where we left off.

Since we've already covered the necessary information about the property, scope of the project, design preferences, budget, timeline, permits, utility modifications, access, and logistics, let's move on to the next question.

Can you please confirm if there's anything else you'd like to add or change about this renovation project that we haven't discussed yet? Any additional details or specific requirements that might impact the project?

(Remember, this is your last chance to add any important details or clarify anything before we finalize the quote.)
yeah just go to town surprise me with this
I see we've already covered a significant amount of information about your project. Let's proceed to the next question.

Can you please confirm if there's anything else you'd like to add or change about this renovation project that we haven't discussed yet? Any additional details or specific requirements that might impact the project?

(Remember, this is your last chance to add any important details or clarify anything before we finalize the quote.)

hehehe""")

In [23]:
client.api_key = random.choice(api_keys)
consultors_list = []
consultationbot = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": system_msg

            },

            {
                "role": "user",
                "content": "You are a proffesional renovation consultor. Based on the chat history, create a streamlined material plan for the user's quote request, choosing what materials to use, and how much would be used and where. MAKE SURE TO BE VERY SPECIFIC in what materials you will use and how much. ALSO MAKE SURE TO PROVIDE A BLURB AT THE START OF THE RESPOnse And THEN A MATERIAL LIST AND HOW MUCH YOU WILL NEEED. DO NOT LIST PRICES JUST LIST MATERIALS NEEDED AND HOW MUCH OF THAT MATERIAL" + "Here is the chat history:" + str(chat_history)

            }
        ],
        model="llama3-70b-8192",
    )
consoltation_output = (consultationbot.choices[0].message.content)
consultors_list.append(consoltation_output)


client.api_key = random.choice(api_keys)
quotebot = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": "You are a helpuful assistant"

            },

            {
                "role": "user",
                "content": "Based on the chat history as well as the consultors list of materials, you must put all the required information for the quote (along with location of property) into a streamlined format so that a web search query can be formed for it." + "Chat History for your own context and info: " + str(chat_history) + "Consultors List: " + str(consultors_list)
            }
        ],
        model="llama3-70b-8192",
    )
streamlined_output = (quotebot.choices[0].message.content)






In [24]:
streamlined_output

'Here is the information organized in a streamlined format for a web search query:\n\n**Location:** 13582 Cobra Drive, Herndon, VA\n\n**Project Details:**\n\n* Residential property\n* Basement renovation (150 sq ft)\n* Premium finishes\n* Modern design style\n* Budget: $500,000\n* Timeline: Completion within 2 months\n\n**Renovation Scope:**\n\n* Add nice wooden flooring\n* Install bright overhead lighting\n* Add shelves lining one side of the wall\n* Create an entertainment space with:\n\t+ 100-inch TV\n\t+ Large couch\n\t+ Games area with pool table and foosball table\n\t+ Tinted glass wall and hot tub\n\n**Utility Modifications:**\n\n* Increase number of vents in the room\n* Run more electricity for appliances\n\n**Access and Logistics:**\n\n* Work hours restricted to 9am-6pm\n* No crane or large trucks allowed\n* Parking constraints in the neighborhood\n\nYou can now use this organized information to create a web search query to find contractors or resources to help with your renov

In [25]:
consoltation_output

"**Material Plan for 150 sq ft Basement Renovation**\n\nBased on our conversation, I've created a comprehensive material plan for your premium basement renovation project. Please find the detailed list of materials needed below:\n\n**Flooring:**\n\n* 150 sq ft of engineered hardwood flooring (e.g., walnut or oak) - 150 sq ft\n* Underlayment material (e.g., foam or cement-based) - 150 sq ft\n* Hardwood flooring adhesive - 1 gallon\n* Wood flooring finish (e.g., polyurethane or oil-based) - 1 gallon\n\n**Walls and Shelves:**\n\n* 100 linear ft of 2x4 lumber for shelving framing - 100 linear ft\n* 100 sq ft of MDF or plywood for shelving - 100 sq ft\n* 1 sheet of 4x8 ft plywood for mirror backing - 1 sheet\n* Wall anchors and screws - 1 box\n* Decorative trim and molding (e.g., crown molding) - 100 linear ft\n\n**Lighting:**\n\n* 10-12 overhead lighting fixtures (e.g., recessed lights or panel lights) - 10-12 units\n* LED light strips for under-shelving lighting - 10-12 ft\n* Lighting con

In [26]:
client.api_key = random.choice(api_keys)
output = agent_executor.invoke({"input":"Given the chat history --> "+streamlined_output+"<-- AS WELL AS THE CONSULTANT'S INFORMATION -->" + consoltation_output + " --> look for labor and material costs for whatever the user asked for in the AREA NEAR ADDRESS OF USERS PROPERTY. maybe also use the costs of houses or properties very near to that location to decide on the cost. BE VERY SPECIFIC. LOTS OF NUMBERS. Also for material costs only use the consoltation_output, and search up the materials individually to find the price."})



> Entering new AgentExecutor chain...
Action:```{"action": "tavily_search_results_json", "action_input": {"query": "basement renovation costs in Herndon, VA"}}```
[{'url': 'https://www.monarchdar.com/service-areas/herndon-va-services/basement-remodeling-herndon/', 'content': 'Remodeling your basement to meet your aesthetic and functional needs is the perfect project for homeowners looking to create their ideal homes. Monarch Design & Remodeling, a basement remodeling company in Herndon, VA, offers custom basement remodeling services. We can handle your entire renovation from beginning to end, from putting together a ...'}, {'url': 'https://www.houzz.com/professionals/basement-remodelers/herndon-va-us-probr0-bo~t_34261~r_4763793', 'content': '11741 Bowman Green Drive, 2nd Floor, Reston, VA 20190. Wilson Enterprise. 5.0 2 Reviews. 1 Hire on Houzz. Locally owned. Remodeling, Reconstruction, and Home Improvement Experts of Fairfax. Staff and crew were professional and very tolerant of so

In [27]:
output

{'input': "Given the chat history --> Here is the information organized in a streamlined format for a web search query:\n\n**Location:** 13582 Cobra Drive, Herndon, VA\n\n**Project Details:**\n\n* Residential property\n* Basement renovation (150 sq ft)\n* Premium finishes\n* Modern design style\n* Budget: $500,000\n* Timeline: Completion within 2 months\n\n**Renovation Scope:**\n\n* Add nice wooden flooring\n* Install bright overhead lighting\n* Add shelves lining one side of the wall\n* Create an entertainment space with:\n\t+ 100-inch TV\n\t+ Large couch\n\t+ Games area with pool table and foosball table\n\t+ Tinted glass wall and hot tub\n\n**Utility Modifications:**\n\n* Increase number of vents in the room\n* Run more electricity for appliances\n\n**Access and Logistics:**\n\n* Work hours restricted to 9am-6pm\n* No crane or large trucks allowed\n* Parking constraints in the neighborhood\n\nYou can now use this organized information to create a web search query to find contractors

In [28]:
output["output"]

'Based on the provided information, I have estimated the labor and material costs for the basement renovation project in Herndon, VA.\n\n**Labor Costs:**\n\n* Basement renovation labor costs in Herndon, VA range from $50,000 to $100,000 or more, depending on the scope of the project (Source: Loudoun County Kitchen Bath and Basement).\n* Average labor cost for a mid-level kitchen remodeling project in the Washington DC/Northern Virginia area is $81,053 (Source: Northwood Construction).\n* Labor costs for basement remodeling in Northern Virginia can be broken down into:\n\t+ Insulation: $2,000 to $4,000\n\t+ Framing: $6,000 to $10,000\n\t+ Drywall: $8,000 to $10,000\n\t+ Waterproofing: $5,000 to $7,000 (Source: Denny & Gardner)\n\n**Material Costs:**\n\n* Engineered hardwood flooring (150 sq ft): $3,000 to $5,000\n* Underlayment material (150 sq ft): $500 to $1,000\n* Hardwood flooring adhesive (1 gallon): $50 to $100\n* Wood flooring finish (1 gallon): $50 to $100\n* 2x4 lumber for shel

In [29]:
refined_output = str(output["output"])
refined_output = refined_output[refined_output.find('"')+1:refined_output.rfind('"')-1]

In [30]:
refined_output

'Based on the provided information, I have estimated the labor and material costs for the basement renovation project in Herndon, VA.\n\n**Labor Costs:**\n\n* Basement renovation labor costs in Herndon, VA range from $50,000 to $100,000 or more, depending on the scope of the project (Source: Loudoun County Kitchen Bath and Basement).\n* Average labor cost for a mid-level kitchen remodeling project in the Washington DC/Northern Virginia area is $81,053 (Source: Northwood Construction).\n* Labor costs for basement remodeling in Northern Virginia can be broken down into:\n\t+ Insulation: $2,000 to $4,000\n\t+ Framing: $6,000 to $10,000\n\t+ Drywall: $8,000 to $10,000\n\t+ Waterproofing: $5,000 to $7,000 (Source: Denny & Gardner)\n\n**Material Costs:**\n\n* Engineered hardwood flooring (150 sq ft): $3,000 to $5,000\n* Underlayment material (150 sq ft): $500 to $1,000\n* Hardwood flooring adhesive (1 gallon): $50 to $100\n* Wood flooring finish (1 gallon): $50 to $100\n* 2x4 lumber for shel

In [31]:
import re

# Original string with escaped characters and formatting issues
original_string = fr''' {refined_output} '''

# Clean up the string
cleaned_string = original_string.replace('\\n', '\n').replace('\\t', '\t').replace(r"\'", "'")

# Optional: further beautify by stripping leading/trailing whitespace and fixing indentation
cleaned_string = re.sub(r'\n\s*', '\n', cleaned_string)  # Remove spaces following new lines
cleaned_string = cleaned_string.strip()  # Remove the leading and trailing whitespace
refined_output = cleaned_string
print(cleaned_string)


Based on the provided information, I have estimated the labor and material costs for the basement renovation project in Herndon, VA.
**Labor Costs:**
* Basement renovation labor costs in Herndon, VA range from $50,000 to $100,000 or more, depending on the scope of the project (Source: Loudoun County Kitchen Bath and Basement).
* Average labor cost for a mid-level kitchen remodeling project in the Washington DC/Northern Virginia area is $81,053 (Source: Northwood Construction).
* Labor costs for basement remodeling in Northern Virginia can be broken down into:
+ Insulation: $2,000 to $4,000
+ Framing: $6,000 to $10,000
+ Drywall: $8,000 to $10,000
+ Waterproofing: $5,000 to $7,000 (Source: Denny & Gardner)
**Material Costs:**
* Engineered hardwood flooring (150 sq ft): $3,000 to $5,000
* Underlayment material (150 sq ft): $500 to $1,000
* Hardwood flooring adhesive (1 gallon): $50 to $100
* Wood flooring finish (1 gallon): $50 to $100
* 2x4 lumber for shelving framing (100 linear ft): $

In [32]:

newstr = ""
ctr = 0
for word in refined_output.split(" "):
    if(ctr % 10 == 0 and ctr != 0):
        newstr += "\n"
    newstr += word + " "
    ctr += 1

print(newstr)

Based on the provided information, I have estimated the labor 
and material costs for the basement renovation project in Herndon, 
VA.
**Labor Costs:**
* Basement renovation labor costs in Herndon, VA range 
from $50,000 to $100,000 or more, depending on the scope 
of the project (Source: Loudoun County Kitchen Bath and Basement).
* 
Average labor cost for a mid-level kitchen remodeling project in 
the Washington DC/Northern Virginia area is $81,053 (Source: Northwood Construction).
* 
Labor costs for basement remodeling in Northern Virginia can be 
broken down into:
+ Insulation: $2,000 to $4,000
+ Framing: $6,000 to 
$10,000
+ Drywall: $8,000 to $10,000
+ Waterproofing: $5,000 to $7,000 (Source: 
Denny & Gardner)
**Material Costs:**
* Engineered hardwood flooring (150 sq ft): 
$3,000 to $5,000
* Underlayment material (150 sq ft): $500 to 
$1,000
* Hardwood flooring adhesive (1 gallon): $50 to $100
* Wood 
flooring finish (1 gallon): $50 to $100
* 2x4 lumber for 
shelving framing (100

In [33]:
client.api_key = random.choice(api_keys)
quotebot2 = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": """You are an expert in PREPARING A REAL ESTATE QUOTE IN PROPER FORMAT from ONLY what is in the user's request, given a web search synthesis as one input and user's request as another input.\
                    YOU ARE TO FOLLOW THIS TEMPLATE AT ALL TIMES - EXACTLY ONLY THIS FORMAT - \
                        ALL COMPONENTS AND ITEMIZED ITEMS SHALL BE LEFT EXACTLY AS IN INPUT (IMPORTANT: ITEMIZED COSTS IN THE INPUT SHALL BE LEFT ALONE; DO NOT LUMP THEM IN MISCELLANEOUS): \
                        **Project Overview:**
- Project Description: [Brief description of the project scope and objectives]
- Length of Time for Project [User's desire for how much time he wants to do renovations]

**Cost Breakdown:**

1. **Labor Costs:**
   - Total Labor Cost: $[Total Labor Cost]
   

2. **Material Costs:**
   - Total Material Cost: $[Total Material Cost]
   - Itemized Costs:
     + [Material 1]: $[Cost]
     + [Material 2]: $[Cost]
     + [Material 3]: $[Cost]
     + [... ALL OTHER MATERIALS MUST FOLLOW THIS FORMAT ]
    

**Total Estimated Cost:**
- Total Cost of Labor: $[Total Labor Cost]
- Total Cost of Materials: $[Total Material Cost]
- **Grand Total: $[Total Estimated Cost]**

**Payment Terms:**
- Deposit Required: [Percentage or fixed amount]
- Payment Schedule: [Details of the payment intervals, e.g., monthly, upon completion of milestones]
- Final Payment Due: [Date or condition upon which the final payment is due]
"""

            },

            {
                "role": "user",
                "content": "Based on the input, which is <<<"+newstr+">>> you must parse the input for the \
                    renovation quote so that your response talks about ONLY the stuff relevant to the user's quote request. \
                        Omit the 'average' costs and 'sources'. IMPORTANT: GIVE AN EXACT AMOUNT FOR MATERIAL VALUE & LABOR COST: NO RANGES ! PICK MAX. \
                            " + "USer's request for your own context and info: " + str(output)
            }
        ],
        model="llama3-70b-8192",
    )
output2 = (quotebot2.choices[0].message.content)
print(output2)

Here is the reformatted real estate quote:

**Project Overview:**
- Project Description: Basement renovation project in Herndon, VA with premium finishes and modern design style.
- Length of Time for Project: Completion within 2 months.

**Cost Breakdown:**

1. **Labor Costs:**
   - Total Labor Cost: $100,000

2. **Material Costs:**
   - Total Material Cost: $74,050
   - Itemized Costs:
     + Engineered hardwood flooring (150 sq ft): $5,000
     + Underlayment material (150 sq ft): $1,000
     + Hardwood flooring adhesive (1 gallon): $100
     + Wood flooring finish (1 gallon): $100
     + 2x4 lumber for shelving framing (100 linear ft): $1,000
     + MDF or plywood for shelving (100 sq ft): $1,000
     + Sheet of 4x8 ft plywood for mirror backing: $100
     + Wall anchors and screws (1 box): $50
     + Decorative trim and molding (100 linear ft): $1,000
     + Overhead lighting fixtures (10-12 units): $1,000
     + LED light strips for under-shelving lighting (10-12 ft): $200
     + 

In [34]:
import re

def parse_quote(quote_str):
    quote_details = {
        "project_description": "",
        "project_duration": "",
        "labor_cost": 0,
        "material_costs": {},
        "total_material_cost": 0,
        "total_estimated_cost": 0,
        "payment_terms": {}
    }

    # Extracting Project Details
    description_match = re.search(r"Project Description: (.+)", quote_str)
    duration_match = re.search(r"Length of Time for Project: (.+)", quote_str)
    if description_match:
        quote_details["project_description"] = description_match.group(1)
    if duration_match:
        quote_details["project_duration"] = duration_match.group(1)

    # Extract labor cost
    labor_cost_match = re.search(r"Total Labor Cost: \$(\d+[\d,]*)", quote_str)
    if labor_cost_match:
        quote_details["labor_cost"] = int(labor_cost_match.group(1).replace(',', ''))

    # Extracting and Summing Material Costs
    material_cost_matches = re.finditer(r"\+\s*(.*?):\s*\$(\d+[\d,]*)", quote_str)
    for match in material_cost_matches:
        item = match.group(1).strip()
        cost = int(match.group(2).replace(',', ''))
        quote_details["material_costs"][item] = cost

    quote_details["total_material_cost"] = sum(quote_details["material_costs"].values())

    # Calculating Total Estimated Cost
    quote_details["total_estimated_cost"] = quote_details["labor_cost"] + quote_details["total_material_cost"]

    # Extracting Payment Terms
    deposit_match = re.search(r"Deposit Required: (\d+)%", quote_str)
    payment_schedule_match = re.search(r"Payment Schedule: Monthly payments of \$(\d+[\d,]*) for (\d+) months", quote_str)
    payment_schedule_match2 = r"Payment Schedule: Monthly payments of \$(\d{1,3}(?:,\d{3})*(?:\.\d{2})?) for (\d+) months"
    final_payment_due_match = re.search(r"Final Payment Due: (.+)", quote_str)
    tmp = None
    if deposit_match:
        deposit_percentage = int(deposit_match.group(1))
        deposit_amount = (quote_details["total_estimated_cost"] * deposit_percentage) / 100
        quote_details["payment_terms"]["deposit_required"] = f"{deposit_percentage}% (${deposit_amount:.2f})"
        tmp = deposit_amount
    if payment_schedule_match or payment_schedule_match2:
        number_of_months = int(re.search(r"for (\d+) months", quote_str).group(1)) if re.search(r"for (\d+) months", quote_str) else 6  # Default to 6 if not found
        monthly_payment = (quote_details["total_estimated_cost"] - tmp) / number_of_months
        quote_details["payment_terms"]["payment_schedule"] = f"Monthly payments of ${int(monthly_payment):,} for {number_of_months} months"

    if final_payment_due_match:
        quote_details["payment_terms"]["final_payment_due"] = final_payment_due_match.group(1)

    return quote_details
import pprint
quote_dict_corrected = parse_quote(output2)
pprint.pprint(quote_dict_corrected)


{'labor_cost': 100000,
 'material_costs': {'100-inch TV': 3000,
                    '2x4 lumber for shelving framing (100 linear ft)': 1000,
                    'Additional vents for increased airflow (2-3 units)': 1000,
                    'Decorative trim and molding (100 linear ft)': 1000,
                    'Ductwork modifications for improved ventilation (1-2 hours of labor)': 1000,
                    'Electrical wire (100 ft of 14-gauge and 50 ft of 12-gauge)': 200,
                    'Engineered hardwood flooring (150 sq ft)': 5000,
                    'Foosball table': 1000,
                    'Hardwood flooring adhesive (1 gallon)': 100,
                    'Hot tub': 5000,
                    'LED light strips for under-shelving lighting (10-12 ft)': 200,
                    'Large couch': 3000,
                    'Lighting control system (optional)': 1000,
                    'MDF or plywood for shelving (100 sq ft)': 1000,
                    'New outlets and switches 

In [35]:
client.api_key = random.choice(api_keys)
corrector = client.chat.completions.create(
        messages=[

            {
                "role": "system",
                "content": "You are an expert in \
                    correcting an input. YOU MUST START THE RESPONSE WITH 'RENOVATION QUOTE' heading"

            },

            {
                "role":"user",
                "content":
                    "Based on the input, which is <<<" + output2 + ">>>, you must parse the input from the renovation quote "
                    "and REPLACE AS IT IS  the total values for the following categories: "
                    "LABOR COSTS: $" + str(quote_dict_corrected["labor_cost"]) + ", "
                    "MATERIAL COSTS: $" + str(quote_dict_corrected["total_material_cost"]) + ", and "
                    "TOTAL ESTIMATED COSTS: $" + str(quote_dict_corrected["total_estimated_cost"]) + ", and "
                     "DEPOSIT: " + str(quote_dict_corrected["payment_terms"]["deposit_required"]) +
                      "MONTHLY PAYMENT " + str(quote_dict_corrected["payment_terms"]["payment_schedule"]) + "." 
            }
        ],
        model="llama3-70b-8192",
    )
corrector_output = (corrector.choices[0].message.content)
print(corrector_output)

RENOVATION QUOTE

**Project Overview:**
- Project Description: Basement renovation project in Herndon, VA with premium finishes and modern design style.
- Length of Time for Project: Completion within 2 months.

**Cost Breakdown:**

1. **Labor Costs:**
   - Total Labor Cost: $100,000

2. **Material Costs:**
   - Total Material Cost: $32,350
   - Itemized Costs:
     + Engineered hardwood flooring (150 sq ft): $5,000
     + Underlayment material (150 sq ft): $1,000
     + Hardwood flooring adhesive (1 gallon): $100
     + Wood flooring finish (1 gallon): $100
     + 2x4 lumber for shelving framing (100 linear ft): $1,000
     + MDF or plywood for shelving (100 sq ft): $1,000
     + Sheet of 4x8 ft plywood for mirror backing: $100
     + Wall anchors and screws (1 box): $50
     + Decorative trim and molding (100 linear ft): $1,000
     + Overhead lighting fixtures (10-12 units): $1,000
     + LED light strips for under-shelving lighting (10-12 ft): $200
     + Lighting control system (o